# 專題（一）：訓練LSTM之歌詞自動填詞器

## 專案目標
- 目標：使用 LSTM 模型去學習五月天歌詞，並且可以自動填詞來產生歌詞
- mayday_lyrics.txt 資料說明：
    - 每一行都是一首歌的歌詞
    - 除去標點符號並以空白表示間格
- 利用 mayday_lyrics.txt 來產生歌詞的序列
- 使用 LSTM 模型去學習歌詞的序列
- 當我們給定開頭的一段歌詞，例如：”給我一首歌”，就可以用 LSTM 猜下一個字，反覆這個過程就可以自動填詞

## 實作提示
- STEP1：從 mayday_lyrics.txt 中取出歌詞
- STEP2：建立每個字的 Index
- STEP3：用 Rolling 的方式打造 LyricsDataset
- STEP4：使用 DataLoader 來包裝 LyricsDataset
- STEP5：建立 LSTM 模型： inputs > nn.Embedding > nn.LSTM > nn.Dropout > 取最後一個 state > nn.Linear > softmax
- STEP6：開始訓練並調整參數
- STEP7：進行 Demo，給定 pre_text ，使用模型迭代的預測下一個字產生歌詞
- (進階) STEP8：在 Demo 時可以採用依照 Softmax 機率來作隨機採樣，這可以增加隨機性，讓歌詞有更多變化，當然你還可以使用機率閥值來避免太奇怪的字出現

## 重要知識點：專題結束後你可以學會
- 如何讀取並處理需要 Rolling 的序列資料
- 了解如何用 Pytorch 建制一個 LSTM 的模型
- 學會如何訓練一個語言模型
- 學會如何隨機抽樣自 Softmax 的分布

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm

In [2]:
# from: https://github.com/gaussic/Chinese-Lyric-Corpus
lyrics_list = [line.strip() for line in open('mayday_lyrics.txt', encoding='utf-8')]
print(lyrics_list[0])

摸不到的顏色 是否叫彩虹 看不到的擁抱 是否叫做微風 一個人 想著一個人 是否就叫寂寞 命運偷走如果 只留下結果 時間偷走初衷 只留下了苦衷 你來過 然後你走後 只留下星空 那一年我們望著星空 有那麼多的燦爛的夢 以為快樂會永久 像不變星空 陪著我 獵戶 天狼 織女光年外沈默 回憶 青春 夢想何時偷偷隕落 我愛過 然後我沈默 人海裡漂流 那一年我們望著星空 未來的未來從沒想過 當故事失去美夢 美夢失去線索 而我們失去聯絡 這一片無言無語星空 為什麼靜靜看我淚流 如果你在的時候 會不會伸手 擁抱我 細數繁星閃爍 細數此生奔波 原來所有 所得 所獲 不如一夜的星空 空氣中的溫柔 回憶你的笑容 徬佛只要伸手 就能觸摸 摸不到的顏色 是否叫彩虹 看不到的擁抱 是否叫做微風 一個人 習慣一個人 這一刻獨自望著星空 從前的從前從沒變過 寂寞可以是忍受 也可以是享受 享受僅有的擁有 那一年我們望著星空 有那麼多的燦爛的夢 至少回憶會永久 像不變星空 陪著我 最後只剩下星空 像不變回憶 陪著我


In [3]:
# 建立詞典對照表
word2index = {}
index2word = {}

i = 0
for words in lyrics_list:
    for word in words:
        if word not in word2index:
            word2index[word] = i
            index2word[i] = word
            i += 1
print(len(word2index))

2101


In [4]:
# 建立數據集
class LyricsDataset(Dataset):
    def __init__(self, lyrics_list, word2index, num_unrollings=10):
        self.word2index = word2index
        self.num_unrollings = num_unrollings
        
        self.gen_rolling_samples(lyrics_list, num_unrollings)
    
    def gen_rolling_samples(self, lyrics_list, num_unrollings):
        self.samples = []
        for lyrics in lyrics_list:
            for i in range(len(lyrics)-self.num_unrollings+1):
                self.samples.append(lyrics[i:i+num_unrollings])
        return self
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        inputs = [self.word2index[i] for i in sample[:-1]]
        targets = [self.word2index[sample[-1]]]

        return torch.LongTensor(inputs), torch.LongTensor(targets)
    def __len__(self):
        return len(self.samples)

In [5]:
batch_size = 128

dataset = LyricsDataset(lyrics_list, word2index)

train_loader = DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True)

In [6]:
# 建立模型
class LM_LSTM(nn.Module):
    def __init__(self, n_hidden, vocab_size, num_layers, dropout_ratio):
        super(LM_LSTM, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, n_hidden)
        self.lstm = nn.LSTM(input_size=n_hidden,
                            hidden_size=n_hidden,
                            num_layers=num_layers,
                            batch_first=True)
        self.dropout = nn.Dropout(dropout_ratio)
        self.fc = nn.Linear(n_hidden, vocab_size)
    def forward(self, inputs):
        
        embedd_inputs = self.embedding(inputs) # [batch, seq_len, hidden]
        out, (hn, cn) = self.lstm(embedd_inputs)  # [batch, seq_len, hidden]
        out = self.dropout(out)  # [batch, seq_len, hidden]
        out = self.fc(out[:,-1,:])  # [batch, hidden]
        return out # [batch, vocab_size]

In [7]:
def train_batch(model, data, criterion, optimizer, device):
    model.train()
    inputs, targets = [d.to(device) for d in data]

    outputs = model(inputs)

    loss = criterion(outputs, targets.squeeze())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

In [8]:
# 訓練模型
epochs = 100
lr = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LM_LSTM(128, len(word2index), 2, 0.2)
model.to(device)

criterion = nn.CrossEntropyLoss(size_average=False)
criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)


for epoch in range(1, 1 + epochs):
    tot_train_loss = 0
    tot_train_count = 0

    for train_data in train_loader:
        loss = train_batch(model, train_data, criterion, optimizer, device)

        tot_train_loss += loss
        tot_train_count += train_data[0].size(0)

    print('epoch ', epoch, 'train_loss: ', tot_train_loss / tot_train_count)

    if epoch % 10 == 0:
        for idx in [0, 50, 99]:
            input_batch = dataset[idx][0].unsqueeze(0).to(device)
            predict = model(input_batch).argmax(dim=-1).item()
            print('Example: "{}"+"{}"'.format(dataset.samples[idx][:-1], index2word[predict]))

C:\Users\Guan-Ting Chen\Anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


epoch  1 train_loss:  5.643004514598451
epoch  2 train_loss:  5.196324539124573
epoch  3 train_loss:  4.86555885632394
epoch  4 train_loss:  4.567099129264321
epoch  5 train_loss:  4.292098639372742
epoch  6 train_loss:  4.03704795566081
epoch  7 train_loss:  3.796037594787044
epoch  8 train_loss:  3.5687444632658725
epoch  9 train_loss:  3.34744874788131
epoch  10 train_loss:  3.139166412772164
Example: "摸不到的顏色 是否"+"不"
Example: " 只留下結果 時間"+"一"
Example: "麼多的燦爛的夢 以"+"為"
epoch  11 train_loss:  2.9442380403473645
epoch  12 train_loss:  2.759889897148055
epoch  13 train_loss:  2.5772094532571113
epoch  14 train_loss:  2.4148118650892614
epoch  15 train_loss:  2.2523768276177734
epoch  16 train_loss:  2.106998293060201
epoch  17 train_loss:  1.975037826106905
epoch  18 train_loss:  1.8484715906927853
epoch  19 train_loss:  1.7249181839199115
epoch  20 train_loss:  1.6136352818783413
Example: "摸不到的顏色 是否"+"叫"
Example: " 只留下結果 時間"+"的"
Example: "麼多的燦爛的夢 以"+"為"
epoch  21 train_loss:  1.514967200

In [9]:
# 模型inference
pre_text = '給我一首歌'
generate_len = 50
prob_threshold = 0.01

result = [word2index[c] for c in pre_text]
for _ in range(generate_len):
    input_example = torch.tensor([result], dtype=torch.long, device=device)
    logit = model(input_example)

    logit = F.softmax(logit, dim=1)
    logit[logit<=prob_threshold] = 0. 
    predict = torch.multinomial(logit, 1).item()

    ## End
    result += [predict]
print(''.join([index2word[i] for i in result]))

給我一首歌 世界曾經人有殘忍 我的每一個眼眶 當純的洗衣機 高中的狂雨 多無聊 原定都再溫柔 誰想念影吧 突然
